## SAS Logs

Reads ETL logs for SAS and SQL sprocs. 
The time calculation for both is the same, but the format and data acquisition is not. 

In [3]:
import glob
import re
from datetime import datetime
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999

In [54]:
username = 'username'
# Load SAS log files into local system
files = glob.glob('/Users/'+username+'/Desktop/Classic_ETL_log_7_8/*.log')
# Eg.files = glob.glob('/Users/UpasanaPattnaik/Desktop/ETL/logs/log_modern/*.log')

In [55]:
# Compatible with MasterETL1.log and MasterETL2.log
files

['/Users/UpasanaPattnaik/Desktop/ETL/Run_4_50_PHC_SAS/sp_SLD_ALL_Hep.log',
 '/Users/UpasanaPattnaik/Desktop/ETL/Run_4_50_PHC_SAS/Drop_Create_Tables.log',
 '/Users/UpasanaPattnaik/Desktop/ETL/Run_4_50_PHC_SAS/MasterETL1.log']

### Format

	Start Time:     07JAN2024:11:08:41
*****The Start codeset*********

	End Time:     07JAN2024:11:08:49
*****The End codeset*********

In [56]:
#Extract logic follows MasterETL1.log and MasterETL2.log pattern
def extract_time(log_path):
    #Regex
    timestamp_pattern = re.compile(r'(\d{2}[a-zA-Z]{3}\d{4}:\d{2}:\d{2}:\d{2})')
    program_pattern = re.compile(r'The (Start|End) (\S[^*]*)')

    log = {'SAS':[], 'Start Time': [], 'End Time':[]}
    sas_dict = {}
    current_timestamp = None
    
    with open(log_path, 'r') as log_file:
        for line in log_file:
            sas_match = re.search(program_pattern, line)
            timestamp_match = re.search(timestamp_pattern, line)

            if timestamp_match:
                timestamp_str = timestamp_match.group(1)
                current_timestamp = datetime.strptime(timestamp_str, '%d%b%Y:%H:%M:%S')
                
            elif sas_match:
                # To account for modern file format cases. 
                if current_timestamp is None:
                    continue
                action, sas_label = sas_match.groups()
                current_sas = sas_label 
                #print(current_timestamp)
                #print("current_sas: ", sas_label)
                
                if action == 'Start':
                    sas_dict[current_sas] = current_timestamp
                elif action == 'End' and current_sas in sas_dict:
                    log['SAS'].append(current_sas)
                    log['Start Time'].append(sas_dict[current_sas])
                    log['End Time'].append(current_timestamp)
            
                    
    df = pd.DataFrame(log)
    df['Time Diff'] = (df['End Time'] - df['Start Time']).dt.total_seconds()
    df['Hours'] = df['Time Diff']//3600
    df['Minutes'] = (df['Time Diff']%3600)//60
    df['Seconds'] = df['Time Diff']%60
    
    return df     

In [57]:
master_etl_log = extract_time(files[2])

In [58]:
master_etl_log

,SAS,Start Time,End Time,Time Diff,Hours,Minutes,Seconds
0,PHCobservations,2024-01-17 15:49:54,2024-01-17 15:49:56,2.0,0.0,0.0,2.0
1,InvestigationDim,2024-01-17 15:49:56,2024-01-17 15:50:06,10.0,0.0,0.0,10.0


In [50]:
master_etl_log['Time Diff'].sum() #//3600

21.0

## SQL Logs

In [59]:
import pyodbc
from sqlalchemy.engine import URL
import sqlalchemy as sa

In [ ]:
host = #Provide host
user = #Provide username
password = #Provide password
database = 'rdb_modern'

In [60]:
host = '10.53.3.146'
user = 'nbs_rdb'
password = 'rdb'
database = 'rdb_modern'

In [61]:
pyodbc.drivers()

['ODBC Driver 17 for SQL Server']

In [62]:
#SQLACLHEMY
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER="+host+";DATABASE="+database+";UID="+user+";PWD="+password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
from sqlalchemy import create_engine
engine = create_engine(connection_url)

In [63]:
with engine.begin() as conn:
    sql_df_og = pd.read_sql_query(sa.text("Select  * from rdb..job_flow_log jfl where batch_id = ******"), conn)
    #sql_df_og = pd.read_sql_query(sa.text("Select  * from rdb_modern..job_flow_log jfl where batch_id = ******"), conn)


In [64]:
sql_df_og

,record_id,batch_id,Dataflow_Name,package_Name,Table_Schema_Name,Step_Number,Step_Name,Column_Name,Status_Type,row_count,Error_Description,Msg_Description1,Msg_Description2,create_dttm,update_dttm
0,12610,240107155340,BATCH_START,sp_nbs_batch_start,None,1.0,start,None,START,0.0,None,None,None,2024-01-07 15:53:40.970,2024-01-07 15:53:40.970
1,12611,240107155340,BATCH_START,sp_nbs_batch_start,None,2.0,Create PHC_UIDS,None,start,4.0,None,None,None,2024-01-07 15:53:41.133,2024-01-07 15:53:41.133
2,12612,240107155340,BATCH_START,nbs_batch_start,None,3.0,Creating PHC_CASE_UIDS,None,start,4.0,None,None,None,2024-01-07 15:53:41.140,2024-01-07 15:53:41.140
3,12613,240107155340,BATCH_START,sp_nbs_batch_start,None,999.0,SP_COMPLETE,None,COMPLETE,4.0,None,None,None,2024-01-07 15:53:43.390,2024-01-07 15:53:43.390
4,12614,240107155340,PATIENT,RDB.CLEAR_PATIENT,None,1.0,SP_Start,None,START,0.0,None,None,None,2024-01-07 15:53:43.407,2024-01-07 15:53:43.407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852,14463,240107155340,INV_SUMM_DATAMART,RDB.INV_SUMM_DATAMART,None,18.0,rdb.dbo.INV_SUMM_DATAMART,None,START,4.0,None,None,None,2024-01-08 13:52:53.917,2024-01-08 13:52:53.917
1853,14464,240107155340,INV_SUMM_DATAMART,RDB.INV_SUMM_DATAMART,None,19.0,INSERTING into INV_SUMM_DATAMART,None,START,0.0,None,None,None,2024-01-08 14:13:05.363,2024-01-08 14:13:05.363
1854,14465,240107155340,INV_SUMM_DATAMART,RDB.INV_SUMM_DATAMART,None,20.0,SP_COMPLETE,None,COMPLETE,0.0,None,None,None,2024-01-08 14:13:05.573,2024-01-08 14:13:05.573
1855,14466,240107155340,BATCH_COMPLETE,nbs_BATCH_COMPLETE,None,1.0,Creating PHC_CASE_UIDS,None,start,1.0,None,None,None,2024-01-08 14:13:05.773,2024-01-08 14:13:05.773


In [65]:
sql_df = sql_df_og

In [66]:
#Custom name updates to standardize logs. Common throughout all ETL runs. 
update_d_labtest = ['D_LabTest', 'RDB.D_LabTest']
condition_d_labtest_1 = sql_df['Dataflow_Name'].isin(update_d_labtest)
condition_d_labtest_2 = sql_df['package_Name'].isin(update_d_labtest)
sql_df.loc[condition_d_labtest_1, 'Dataflow_Name'] = 'D_LABTEST'
sql_df.loc[condition_d_labtest_2, 'package_Name'] = 'RDB.D_LABTEST'

update_Vaccination_Record = ['Vaccination_Record']
condition_Vaccination_Record = sql_df['Dataflow_Name'].isin(update_Vaccination_Record)
sql_df.loc[condition_Vaccination_Record, 'Dataflow_Name'] = 'VaccinationRecord'

update_Lab100 = ['Lab100','RDB.D_LAB100']
condition_Lab100_1 = sql_df['Dataflow_Name'].isin(update_Lab100)
condition_Lab100_2 = sql_df['package_Name'].isin(update_Lab100)
sql_df.loc[condition_Lab100_1, 'Dataflow_Name'] = 'D_LAB100'
sql_df.loc[condition_Lab100_2, 'Dataflow_Name'] = 'LAB100_DATAMART'
sql_df.loc[condition_Lab100_2, 'package_Name'] = 'RDB.LAB100_DATAMART'

update_EVENT_METRIC_DATAMART= ['EVENT_METRIC_INC DATAMART', 'RDB.dbo.EVENT_METRIC_INC']
condition_EVENT_METRIC_DATAMART_1 = sql_df['Dataflow_Name'].isin(update_EVENT_METRIC_DATAMART)
condition_EVENT_METRIC_DATAMART_2 = sql_df['package_Name'].isin(update_EVENT_METRIC_DATAMART)
sql_df.loc[condition_EVENT_METRIC_DATAMART_1, 'Dataflow_Name'] = 'EVENT_METRIC DATAMART'
sql_df.loc[condition_EVENT_METRIC_DATAMART_2, 'package_Name'] = 'RDB.dbo.EVENT_METRIC'

update_LAB101_DATAMART = ['D_LAB101', 'RDB.D_LAB101']
condition_LAB101_DATAMART_1 = sql_df['Dataflow_Name'].isin(update_LAB101_DATAMART)
condition_LAB101_DATAMART_2 = sql_df['package_Name'].isin(update_LAB101_DATAMART)
sql_df.loc[condition_LAB101_DATAMART_1, 'Dataflow_Name'] = 'LAB101_DATAMART'
sql_df.loc[condition_LAB101_DATAMART_2, 'package_Name'] = 'RDB.LAB101_DATAMART'

update_nbs_batch_start = ['nbs_batch_start']
condition_nbs_batch_start = sql_df['package_Name'].isin(update_nbs_batch_start)
sql_df.loc[condition_nbs_batch_start, 'package_Name'] = 'sp_nbs_batch_start'

update_nbs_batch_complete = ['sp_nbs_batch_complete']
condition_nbs_batch_complete = sql_df['package_Name'].isin(update_nbs_batch_complete)
sql_df.loc[condition_nbs_batch_complete, 'package_Name'] = 'nbs_BATCH_COMPLETE'

update_F_STD_PAGE_CASE = ['RDB.S_F_STD_PAGE_CASE', 'S_F_STD_PAGE_CASE', 'RDB.F_STD_PAGE_CASE']
condition_F_STD_PAGE_CASE = sql_df['package_Name'].isin(update_F_STD_PAGE_CASE)
sql_df.loc[condition_F_STD_PAGE_CASE, 'package_Name'] = 'F_STD_PAGE_CASE'

update_F_PAGE_CASE = ['RDB.S_F_PAGE_CASE', 'S_F_PAGE_CASE', 'RDB.F_PAGE_CASE']
condition_F_PAGE_CASE = sql_df['package_Name'].isin(update_F_PAGE_CASE)
sql_df.loc[condition_F_PAGE_CASE, 'package_Name'] = 'F_PAGE_CASE'

update_INVESTIGATION_REPEAT_1 = ['RDB.D_INVESTIGATION_REPEAT']
update_INVESTIGATION_REPEAT_2 = ['D_INVESTIGATION_REPEAT']
condition_INVESTIGATION_REPEAT_1 = sql_df['package_Name'].isin(update_INVESTIGATION_REPEAT_1)
condition_INVESTIGATION_REPEAT_2 = sql_df['package_Name'].isin(update_INVESTIGATION_REPEAT_2)
sql_df.loc[condition_INVESTIGATION_REPEAT_1, 'package_Name'] = 'RDB.SLD_INVESTIGATION_REPEAT'
sql_df.loc[condition_INVESTIGATION_REPEAT_2, 'package_Name'] = 'SLD_INVESTIGATION_REPEAT'

update_STD_HIV_DATAMART_1 = ['RDB.INV_HIV']
update_STD_HIV_DATAMART_2 = ['INV_HIV']
condition_STD_HIV_DATAMART_1 = sql_df['package_Name'].isin(update_STD_HIV_DATAMART_1)
condition_STD_HIV_DATAMART_2 = sql_df['package_Name'].isin(update_STD_HIV_DATAMART_2)
sql_df.loc[condition_STD_HIV_DATAMART_1, 'package_Name'] = 'RDB.STD_HIV_DATAMART'
sql_df.loc[condition_STD_HIV_DATAMART_2, 'package_Name'] = 'STD_HIV_DATAMART'

update_INV_SUMM_DATAMART_1 = ['RDB.D_INV_Summ_DataMart']
update_INV_SUMM_DATAMART_2 = ['D_INV_Summ_DataMart']
condition_INV_SUMM_DATAMART_1 = sql_df['package_Name'].isin(update_INV_SUMM_DATAMART_1)
condition_INV_SUMM_DATAMART_2 = sql_df['package_Name'].isin(update_INV_SUMM_DATAMART_2)
sql_df.loc[condition_INV_SUMM_DATAMART_1, 'package_Name'] = 'RDB.INV_SUMM_DATAMART'
sql_df.loc[condition_INV_SUMM_DATAMART_2, 'package_Name'] = 'INV_SUMM_DATAMART'

In [67]:
sql_df = sql_df.sort_values(by=['Dataflow_Name', 'package_Name','Step_Number'])

In [68]:
def agg_group_mod(send_over):
    step1_cdttm = send_over.loc[send_over['Step_Number'].isin([0,1]),'create_dttm'].min()
    last_step = send_over.loc[send_over['Step_Number'] == send_over['Step_Number'].max(), 'update_dttm'].max()
    return pd.Series(
        {
            'step1_cdttm': step1_cdttm,
            'end_time': last_step,
        }
    )

In [69]:
result_df_up = sql_df.groupby(['Dataflow_Name', 'package_Name']).apply(agg_group_mod).reset_index()

In [70]:
result_df_up['time_diff_seconds'] = (result_df_up['end_time'] - result_df_up['step1_cdttm']).dt.total_seconds()
result_df_up['Hours'] = result_df_up['time_diff_seconds']//3600
result_df_up['Minutes'] = (result_df_up['time_diff_seconds']%3600)//60
result_df_up['Seconds'] = result_df_up['time_diff_seconds']%60

In [71]:
display(result_df_up.sort_values(by='step1_cdttm'))

,Dataflow_Name,package_Name,step1_cdttm,end_time,time_diff_seconds,Hours,Minutes,Seconds
1,BATCH_START,sp_nbs_batch_start,2024-01-07 15:53:40.970,2024-01-07 15:53:43.390,2.420,0.0,0.0,2.420
117,PATIENT,RDB.CLEAR_PATIENT,2024-01-07 15:53:43.407,2024-01-07 15:53:52.530,9.123,0.0,0.0,9.123
118,SP_PRE_RUN,RDB.SP_PRE_RUN,2024-01-07 15:53:52.740,2024-01-07 16:04:12.050,619.310,0.0,10.0,19.310
11,D_PATIENT,RDB.D_PATIENT,2024-01-07 16:04:12.290,2024-01-07 16:05:34.610,82.320,0.0,1.0,22.320
12,D_PROVIDER,RDB.D_PROVIDER,2024-01-07 16:05:38.070,2024-01-07 16:06:32.923,54.853,0.0,0.0,54.853
10,D_ORGANIZATION,RDB.D_ORGANIZATION,2024-01-07 16:06:32.963,2024-01-07 16:06:50.927,17.964,0.0,0.0,17.964
13,EVENT_METRIC DATAMART,RDB.dbo.EVENT_METRIC,2024-01-07 16:06:51.000,2024-01-07 16:08:40.960,109.960,0.0,1.0,49.960
121,USER_PROFILE,RDB.CLEAR_USER_PROFILE,2024-01-07 16:08:40.970,2024-01-07 16:08:40.970,0.000,0.0,0.0,0.000
122,User_Profile,RDB.User_Profile,2024-01-07 16:08:40.983,2024-01-07 16:08:41.073,0.090,0.0,0.0,0.090
120,S_LDF_DIMENSIONAL_DATA,RDB.S_LDF_DIMENSIONAL_DATA,2024-01-08 13:27:19.850,2024-01-08 13:27:51.580,31.730,0.0,0.0,31.730


In [72]:
#Compare
display(result_df_up.sort_values(by='step1_cdttm'))

,Dataflow_Name,package_Name,step1_cdttm,end_time,time_diff_seconds,Hours,Minutes,Seconds
1,BATCH_START,sp_nbs_batch_start,2024-01-07 15:53:40.970,2024-01-07 15:53:43.390,2.420,0.0,0.0,2.420
117,PATIENT,RDB.CLEAR_PATIENT,2024-01-07 15:53:43.407,2024-01-07 15:53:52.530,9.123,0.0,0.0,9.123
118,SP_PRE_RUN,RDB.SP_PRE_RUN,2024-01-07 15:53:52.740,2024-01-07 16:04:12.050,619.310,0.0,10.0,19.310
11,D_PATIENT,RDB.D_PATIENT,2024-01-07 16:04:12.290,2024-01-07 16:05:34.610,82.320,0.0,1.0,22.320
12,D_PROVIDER,RDB.D_PROVIDER,2024-01-07 16:05:38.070,2024-01-07 16:06:32.923,54.853,0.0,0.0,54.853
10,D_ORGANIZATION,RDB.D_ORGANIZATION,2024-01-07 16:06:32.963,2024-01-07 16:06:50.927,17.964,0.0,0.0,17.964
13,EVENT_METRIC DATAMART,RDB.dbo.EVENT_METRIC,2024-01-07 16:06:51.000,2024-01-07 16:08:40.960,109.960,0.0,1.0,49.960
121,USER_PROFILE,RDB.CLEAR_USER_PROFILE,2024-01-07 16:08:40.970,2024-01-07 16:08:40.970,0.000,0.0,0.0,0.000
122,User_Profile,RDB.User_Profile,2024-01-07 16:08:40.983,2024-01-07 16:08:41.073,0.090,0.0,0.0,0.090
120,S_LDF_DIMENSIONAL_DATA,RDB.S_LDF_DIMENSIONAL_DATA,2024-01-08 13:27:19.850,2024-01-08 13:27:51.580,31.730,0.0,0.0,31.730


In [73]:
result_df_up['Seconds'].sum()

508.97599999999983

In [85]:
result_df_up.sort_values(by='step1_cdttm').to_csv('job_flow_log_7th.csv')
